In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

In [65]:
data_set = pd.read_csv('banco_de_dados.csv')

data_set.columns = ['data', 'eletrodomesticos', 'luzes', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'consumo_de_energia', 'rv2', 'NSM', 'Status_semana',
       'dia_da_semana']

data_set.drop(['Press_mm_hg', 'Windspeed', 'Visibility', 'rv2', 'NSM', 'Status_semana'],axis=1, inplace=True)

'Tdewpoint'--> Ponto de orvalho é uma medida da umidade atmosférica. É a temperatura à qual o ar deve ser resfriado para atingir a saturação (assumindo que a pressão do ar e o teor de umidade são constantes). Um ponto de orvalho mais alto indica mais umidade presente no ar.

Medidas do consumo de energia a cada 10 minutos e medidas dos sensores de temperatura e umidade a cada 3 minutos, sendo assim necessario uma interpolação para pular as medições dos sensores no intervalo de 3 medições

In [66]:
intervalo = 3
data_set = data_set.iloc[::intervalo]
data_set

,data,eletrodomesticos,luzes,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,RH_7,T8,RH_8,T9,RH_9,T_out,RH_out,Tdewpoint,consumo_de_energia,dia_da_semana
0,2016-01-11 17:00:00,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.00,...,41.626667,18.200000,48.900000,17.033333,45.530000,6.600000,92.000000,5.300000,13.275433,Monday
3,2016-01-11 17:10:00,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.00,...,41.560000,18.200000,48.863333,17.066667,45.560000,6.483333,92.000000,5.200000,18.606195,Monday
6,2016-01-11 17:40:00,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.89,...,41.230000,18.100000,48.590000,17.000000,45.400000,6.133333,92.000000,4.900000,10.084097,Monday
9,2016-01-11 17:50:00,50,40,19.890000,46.026667,19.200000,44.500000,19.790000,44.933333,18.89,...,41.260000,18.100000,48.590000,17.000000,45.290000,6.016667,92.000000,4.800000,44.919484,Monday
12,2016-01-11 18:20:00,60,40,19.790000,45.597500,19.200000,44.433333,19.730000,44.790000,18.89,...,41.400000,18.100000,48.590000,17.000000,45.290000,5.933333,91.666667,4.666667,31.455702,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29592,2016-05-27 16:40:00,180,0,25.500000,46.530000,26.000000,41.725714,27.856667,40.500000,24.70,...,44.634000,24.434000,51.632000,23.200000,46.652857,22.933333,55.000000,13.333333,25.296732,Friday
29595,2016-05-27 16:50:00,120,0,25.500000,47.456667,26.000000,42.320000,27.663333,40.693333,24.70,...,44.668571,24.540000,51.500000,23.200000,46.718000,22.966667,55.000000,13.366667,1.796395,Friday
29598,2016-05-27 17:10:00,90,0,25.533333,46.860000,25.978000,42.534000,27.323333,41.090000,24.70,...,44.764286,24.666667,50.445000,23.200000,46.745000,22.866667,55.333333,13.366667,16.966162,Friday
29601,2016-05-27 17:30:00,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.70,...,44.414286,24.700000,49.790000,23.200000,46.790000,22.600000,56.000000,13.300000,49.282940,Friday


In [67]:
# Divisão dos dados em conjuntos de treinamento e teste
train_size = 0.7  # 70% para treinamento, 30% para teste
train, test = train_test_split(data_set, train_size=train_size, shuffle=False)

In [68]:
def create_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        target = data[i+sequence_length]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

In [69]:
# comprimento da sequência temporal
sequence_length = 10  # 10 leituras anteriores para prever a próxima

X_train, y_train = create_sequences(train[['eletrodomesticos', 'luzes', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out',]].values, sequence_length)
X_test, y_test = create_sequences(test[['eletrodomesticos', 'luzes', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out',]].values, sequence_length)

In [74]:
model = Sequential()
model.add(LSTM(50, input_shape=(sequence_length, 21)))  # 2 porque temos 2 recursos de entrada (temperatura e umidade)
model.add(Dense(1))  # Camada de saída com uma única saída (previsão de consumo de energia)

# Compile o modelo
model.compile(optimizer='adam', loss='mean_squared_error')  # Pode ajustar a função de perda 

epochs: É o número de épocas de treinamento, ou seja, quantas vezes o modelo passará por todo o conjunto de treinamento. Cada época é uma iteração completa pelos dados de treinamento.

batch_size: É o tamanho do lote (batch) usado no treinamento. Em vez de atualizar os pesos do modelo após cada exemplo, o treinamento é feito em lotes de tamanho batch_size. Isso é mais eficiente em termos computacionais. O tamanho do lote afeta a convergência do modelo e a velocidade de treinamento.

In [76]:
model.fit(X_train, y_train, epochs=50, batch_size=32)  # número de épocas e tamanho 

Epoch 1/50
216/216 [==============================] - 1s 4ms/step - loss: 1072.4070
Epoch 2/50
216/216 [==============================] - 1s 3ms/step - loss: 1072.2838
Epoch 3/50
216/216 [==============================] - 1s 3ms/step - loss: 1072.0908
Epoch 4/50
216/216 [==============================] - 1s 3ms/step - loss: 1072.2394
Epoch 5/50
216/216 [==============================] - 1s 4ms/step - loss: 1072.0503
Epoch 6/50
216/216 [==============================] - 1s 3ms/step - loss: 1071.9941
Epoch 7/50
216/216 [==============================] - 1s 3ms/step - loss: 1072.0774
Epoch 8/50
216/216 [==============================] - 1s 4ms/step - loss: 1071.9666
Epoch 9/50
216/216 [==============================] - 1s 4ms/step - loss: 1071.9960
Epoch 10/50
216/216 [==============================] - 1s 4ms/step - loss: 1071.8660
Epoch 11/50
216/216 [==============================] - 1s 4ms/step - loss: 1072.0072
Epoch 12/50
216/216 [==============================] - 1s 4ms/step - loss:

In [77]:
loss = model.evaluate(X_test, y_test)
print(f'Loss no conjunto de teste: {loss}')

93/93 [==============================] - 0s 2ms/step - loss: 759.7043
Loss no conjunto de teste: 759.704345703125


model.evaluate(X_test, y_test): Esta linha de código avalia o modelo usando o conjunto de teste. X_test são as características de teste e y_test são os rótulos de teste (os valores reais que você está tentando prever).

A função evaluate calcula a perda com base nas previsões do modelo para o conjunto de teste e nos rótulos reais. A perda é uma medida que indica o quão bem as previsões do modelo estão se aproximando dos valores reais.

O resultado da função evaluate é armazenado na variável loss. Essa variável conterá o valor da perda no conjunto de teste.